In [ ]:
!pip install tweet-preprocessor

In [1]:
import pandas as pd
import re, string
import numpy as np

In [2]:
import random, torch

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [3]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [4]:
path_train = "/kaggle/input/hope-task-2-english/Task 2 Training set (English).csv"
path_val = "/kaggle/input/hope-task-2-english/val.csv"

# Read dataset

In [5]:
import string,re
def preprocessing_text(text):
    text = p.clean(text)

    text = text.strip()
    text = text.translate(text.maketrans('', '', string.punctuation.replace("_","")))
    text = re.sub('\\s+',' ',text).strip()
    return text

In [6]:
import pandas as pd

def convert_label(text):
    if text == "Not Hope":
        return 0
    elif text == "Generalized Hope":
        return 1
    elif text == "Unrealistic Hope":
        return 2
    elif text == "Realistic Hope":
        return 3
    else:
        print("Error: ", text)
        return 0
    

def read_and_preprocessing(path_data):
    df = pd.read_csv(path_data)
    df["multiclass"] = df["multiclass"].apply(convert_label)
    x_input = df["text"].apply(preprocessing_text).tolist()
    y_output = df["multiclass"].tolist()
    ids = df["id"].tolist()
    return x_input,y_output,ids

train_texts, train_labels,train_ids = read_and_preprocessing(path_train)
valid_texts,valid_labels,valid_ids = read_and_preprocessing(path_val)
print(len(train_texts),len(train_labels))
print(len(valid_texts),len(valid_labels))

6192 6192
1032 1032


In [7]:
df_train = pd.DataFrame(list(zip(train_texts, train_labels)),
               columns =['x_data', 'y_output'])
df_train.head()

,x_data,y_output
0,im really liking this project lets work togeth...,3
1,oh shit really i would hope theyd shed some mo...,1
2,good morning bud smiling face with hearts anot...,1
3,i aspire to have the level of delusion to beli...,2
4,projects are continuously attacked by hackers ...,0


# Build Filter Classifier

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "microsoft/deberta-v3-base" #Try different models here 

bert_model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=4)
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=512)

In [9]:
max_length = 512
train_encodings = tokenizer(train_texts, truncation=True, max_length=max_length, padding=True)

In [10]:
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels)

In [11]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=10,              # total number of training epochs
    learning_rate=5e-5,
    per_device_train_batch_size=32,  # batch size per device during training
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,
    save_total_limit = 1
)
 
trainer = Trainer(
    model=bert_model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
)
trainer.train()
# API: fac85ccacc3dffb183116aba932e6bcc08010443

2024-03-28 17:33:26.844921: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-28 17:33:26.845040: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-28 17:33:26.959196: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_con

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 978.00 MiB. GPU 0 has a total capacty of 15.89 GiB of which 270.12 MiB is free. Process 2324 has 15.63 GiB memory in use. Of the allocated memory 7.44 GiB is allocated by PyTorch, and 7.89 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# Inference

In [ ]:
def make_prediction(review,tokenizer,trainer):
    demo_input = preprocessing_text(review)
    demo_encodings = tokenizer([demo_input], truncation=True, max_length = max_length, padding=True)
    test_dataset = CustomDataset(demo_encodings, [0])
    predic_demo = trainer.predict(test_dataset)[0]
    predict_label = np.argmax(predic_demo, axis=1).flatten().tolist()[0]
    return predict_label

In [ ]:
y_pred = []

for review in valid_texts:
    a = make_prediction(review,tokenizer,trainer)
    y_pred.append(a)
print(y_pred[:10])
print(valid_labels[:10])

In [ ]:
from sklearn.metrics import *

print("M_Pr: ", round(precision_score(valid_labels, y_pred, average='macro'),4))
print("M_Re: ", round(recall_score(valid_labels, y_pred, average='macro'),4))
print("M_F1: ", round(f1_score(valid_labels, y_pred, average='macro'),4))

print("W_Pr: ", round(precision_score(valid_labels, y_pred, average='weighted'),4))
print("W_Re: ", round(recall_score(valid_labels, y_pred, average='weighted'),4))
print("W_F1: ", round(f1_score(valid_labels, y_pred, average='weighted'),4))

print("acc:", round(accuracy_score(valid_labels, y_pred), 4))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(valid_labels, y_pred))

# Submission for Validation datasets

In [ ]:
# def convert2category(y_pred):
#     y_label = []
#     for y in y_pred:
#         if y == 0:
#               y_label.append("Not Hope")
#         elif y == 1:
#               y_label.append("Generalized Hope")
#         elif y == 2:
#               y_label.append("Unrealistic Hope")
#         elif y == 3:
#               y_label.append("Realistic Hope")
#     return y_label

In [ ]:
# name_sub = "predictions.csv"
# y_pred_label = convert2category(y_pred)
# df_sub = pd.DataFrame(list(zip(valid_ids, y_pred_label)),
#                columns =['id', 'category'])
# df_sub.to_csv(name_sub)
# df_sub.head()

# Submission for Test datasets

In [ ]:
def read_and_preprocessing_for_test(path_data):
    df = pd.read_csv(path_data)
    x_input = df["text"].apply(preprocessing_text).tolist()
    ids = df["id"].tolist()
    return x_input,ids

test_texts,test_ids = read_and_preprocessing_for_test(path_test)
print(len(test_texts))

def make_prediction(review,tokenizer,trainer):
    demo_input = preprocessing_text(review)
    demo_encodings = tokenizer([demo_input], truncation=True, max_length = max_length, padding=True)
    test_dataset = CustomDataset(demo_encodings, [0])
    predic_demo = trainer.predict(test_dataset)[0]
    predict_label = np.argmax(predic_demo, axis=1).flatten().tolist()[0]
    return predict_label

In [ ]:
y_pred_test = []

for review in test_texts:
    a = make_prediction(review,tokenizer,trainer)
    y_pred_test.append(a)

In [ ]:
def convert2category(y_pred):
    y_label = []
    for y in y_pred:
        if y == 0:
              y_label.append("Not Hope")
        else:
              y_label.append("Hope")
    return y_label

In [ ]:
name_sub = "predictions.csv"
y_pred_label = convert2category(y_pred_test)
df_sub = pd.DataFrame(list(zip(test_ids, y_pred_label)),
               columns =['id', 'category'])
df_sub.to_csv(name_sub)
df_sub.head()